***GENERATED CODE FOR patientchargesregression PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class DBFSConnector:

    def fetch(spark, config):
        df = spark.read.\
            options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                    inferschema='true',
                    delimiter=eval(config)["delimiter"])\
            .csv(eval(config)['url'])
        display(df.limit(2).toPandas())
        return df

    def put(df, path):
        df.to_csv("/dbfs" + eval(config)['url'], header=True, index=False)
        return True


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def replaceByMean(self, feature, df, mean_=-1):

        meanValue = df.select(mean(col(feature.name)).alias(
            'mean')).collect()[0]["mean"]
        df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                                         meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        maxValue = df.select(max(col(feature.name)).alias('max')).collect()[
            0]["max"]
        df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        minValue = df.select(min(col(feature.name)).alias('min')).collect()[
            0]["min"]
        df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        stddevValue = df.select(stddev(col(feature.name)).alias(
            'stddev')).collect()[0]["stddev"]
        df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        fillValue = df.where(col(feature.name).isNotNull()
                             ).head(1)[0][feature.name]
        df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'transformation_label': 'String Indexer', 'feature_label': 'sex'}], 'feature': 'sex', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '26', 'mean': '', 'stddev': '', 'min': 'female', 'max': 'male', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'sex'}, {'transformation_label': 'String Indexer', 'feature_label': 'sex'})
        transformationDF = transformationDF.drop('sex')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'transformation_label': 'String Indexer', 'feature_label': 'smoker'}], 'feature': 'smoker', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '26', 'mean': '', 'stddev': '', 'min': 'no', 'max': 'yes', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'smoker'}, {'transformation_label': 'String Indexer', 'feature_label': 'smoker'})
        transformationDF = transformationDF.drop('smoker')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'transformation_label': 'String Indexer', 'feature_label': 'region'}], 'feature': 'region', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '26', 'mean': '', 'stddev': '', 'min': 'northeast', 'max': 'southwest', 'missing': '0', 'distinct': '4'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'region'}, {'transformation_label': 'String Indexer', 'feature_label': 'region'})
        transformationDF = transformationDF.drop('region')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import pyspark


def linear_regression(sparkDF, labels, features):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = (sparkDF.toPandas())
    lr_model = LinearRegression()
    X_train, X_test, y_train, y_test = train_test_split(
        df[features], df[labels])
    lr_model.fit(X_train, y_train)
    display(" Accuracy of Model : %s" % lr_model.score(X_test, y_test))

    data = {'model': lr_model,
            'X_test': X_test,
            'y_test': y_test,
            'label': labels,
            'columnNames': df.columns}
    return data


***READING DATAFRAME***

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

#%run patientchargesregressionHooks.ipynb
try:
	#sourcePreExecutionHook()

	inurance = DBFSConnector.fetch(spark, "{'url': '/FileStore/platform/uploadedSourceFiles/1634804841530_insurance.csv', 'filename': '1634804841530_insurance.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'dbfs_token': '', 'dbfs_domain': '', 'FilePath': '/Healthcare Models/Patient Charges - Regression/insurance.csv', 'viewFileName': 'insurance.csv', 'is_header': 'Use Header Line', 'server_url': '/numtraPlatform/NumtraPlatformV2/uploads/platform/', 'results_url': 'http://52.142.62.122:4040/api/read/hdfs'}")
	#sourcePostExecutionHook(inurance)

except Exception as ex: 
	logging.error(ex)


***TRANSFORMING DATAFRAME***

In [ ]:
#%run patientchargesregressionHooks.ipynb
try:
	#transformationPreExecutionHook()

	manualfe = TransformationMain.run(inurance,json.dumps( {"FE": [{"transformationsData": [{"transformation_label": "novalue"}], "feature": "age", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "26", "mean": "43.0", "stddev": "14.97", "min": "18", "max": "63", "missing": "0"}, "updatedLabel": "age"}, {"transformationsData": [{"transformation_label": "String Indexer", "feature_label": "sex"}], "feature": "sex", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "26", "mean": "", "stddev": "", "min": "female", "max": "male", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "sex"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "bmi", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "26", "mean": "29.8", "stddev": "6.27", "min": "18.3", "max": "47.74", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "bmi"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "children", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "26", "mean": "1.12", "stddev": "1.37", "min": "0", "max": "5", "missing": "0"}, "updatedLabel": "children"}, {"transformationsData": [{"transformation_label": "String Indexer", "feature_label": "smoker"}], "feature": "smoker", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "26", "mean": "", "stddev": "", "min": "no", "max": "yes", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "smoker"}, {"transformationsData": [{"transformation_label": "String Indexer", "feature_label": "region"}], "feature": "region", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "26", "mean": "", "stddev": "", "min": "northeast", "max": "southwest", "missing": "0", "distinct": "4"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "region"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "charges", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "26", "mean": "13907.16", "stddev": "11201.74", "min": "1909.52745", "max": "41676.0811", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "charges"}]}))

	#transformationPostExecutionHook(manualfe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run patientchargesregressionHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=linear_regression(manualfe, ["bmi", "children", "charges"], "age")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)
